In [0]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [34]:
data=pd.read_csv('breast-cancer-wisconsin_.data', names=["id", "clump_thickness", "cell_size", "cell_shape", "margin_adhesion", "ep_cell_size", "bare_nuc", "bland_chromatin", "normal_nuc", "mitoses", "class"])
data.apply(pd.to_numeric)
clump_thickness=data['clump_thickness']
cell_size=data['cell_size']
cell_shape=data['cell_shape']
margin_adhesion=data['margin_adhesion']
ep_cell_size=data['ep_cell_size']
bare_nuc=data['bare_nuc']
bland_chromatin=data['bland_chromatin']
normal_nuc=data['normal_nuc']
mitoses=data['mitoses']

y=data['class']
x=np.column_stack((clump_thickness, cell_size, cell_shape, margin_adhesion, ep_cell_size, bare_nuc, bland_chromatin, normal_nuc, mitoses))
x=sm.add_constant(x, prepend=True)

bin_model=sm.GLM(y, x, family=sm.families.NegativeBinomial())
bin_results=bin_model.fit()
print(bin_results.summary())
print('Odds Ratio: ', str(np.exp(bin_results.params)))
print()
print('Parameters: ', str(bin_results.params))
print()
bin_param=bin_results.params
confidence=bin_results.conf_int()
confidence['OR']=bin_param
confidence.columns=['2.5%', '97.5%', 'OR']
print(np.exp(confidence))
print()
print('----------------------')
print()

gamma_model=sm.GLM(y, x, family=sm.families.Gamma())
gamma_results=gamma_model.fit()
print(gamma_results.summary())
print('Odds Ratio: ', str(np.exp(gamma_results.params)))
print()
print('Parameters: ', str(gamma_results.params))
print()
gamma_param=gamma_results.params
confidence=gamma_results.conf_int()
confidence['OR']=gamma_param
confidence.columns=['2.5%', '97.5%', 'OR']
print(np.exp(confidence))
print()
print('----------------------')
print()

gauss_model=sm.GLM(y, x, family=sm.families.Gaussian())
gauss_results=gauss_model.fit()
print(gauss_results.summary())
print('Odds Ratio: ', str(np.exp(gauss_results.params)))
print()
print('Parameters: ', str(gauss_results.params))
print()
gauss_param=gauss_results.params
confidence=gauss_results.conf_int()
confidence['OR']=gauss_param
confidence.columns=['2.5%', '97.5%', 'OR']
print(np.exp(confidence))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  class   No. Observations:                  683
Model:                            GLM   Df Residuals:                      673
Model Family:        NegativeBinomial   Df Model:                            9
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1450.3
Date:                Tue, 29 Oct 2019   Deviance:                       8.9825
Time:                        15:37:02   Pearson chi2:                     9.52
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5246      0.102      5.131      0.0

/usr/local/lib/python3.6/dist-packages/statsmodels/genmod/generalized_linear_model.py:273: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


Analysis:

I have trained three separate GLM models with one discrete family (Negative Binomial) and two continuous families (Gamma and Gaussian). The different link functions are as specified:

- Gamma: Inverse Power
- Gaussian: Identity
- Negative Binomial: Log 

The summaries from the different GLM models are as shown above. The explanations derived from each different GLM for the odds ratio and the CI values are:

Gaussian:

We notice that all the independent variables have positive (greater than 1). This implies that the dependent variable is strongly correlated with the parameters for all the independent variables. The dependent variable is most influenced by variable x5 due to its highest odds ratio. We also see that it has a log likelihood of -303 and Pearson chi2 of 97.4 which indicates a decent fit of the model. We also see the statistical significance levels of the parameters at the 5% significance level.

Gamma:

We notice that many independent variables have positive (less than 1). This implies that the dependent variable is weakly correlated with the independent variables for whom the parameter values are less than 1. The dependent variable is most influenced by variable x9 and has a strong correlation with it due to its high odds ratio. We also see that it has a log likelihood of -319 and Pearson chi2 of 17.2 which indicates a good fit of the model.

Negative Binomial:

We notice that many independent variables have positive (greater than 1). This implies that the dependent variable is correlated well with the parameters of the independent variables. However, we see that x9 has an odds ratio less than 1, hence it is weakly correlated (negative) with the model. The dependent variable is most influenced by variable x1 and has a strong correlation with it due to its high odds ratio. We also see that it has a log likelihood of -1450 and Pearson chi2 of 9.1 which indicates a very good fit of the model.

Also, on doing a comparative analysis we see that the best fit GLM is obtained on taking the Negative Binomial family with the log link function. This is because clearly our data's y is distributed as a discrete distribution since it is essentially a classification task. The continuous functions are not able to generalise well to the discrete function and overfit the data in a sense, hence produces poorer GLMs.
